<a href="https://colab.research.google.com/github/JJader/api-frontend/blob/main/notebook/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurando o ambiente

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U pandas==1.5.3 &> /dev/null
!pip install -q findspark pyspark "mlflow==2.15.1" &> /dev/null

In [4]:
import os
import findspark

In [5]:
# instalar as dependências do spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# !wget  https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf /content/drive/MyDrive/Colab\ Notebooks/spark-3.1.2-bin-hadoop2.7.tgz

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
findspark.init()

In [6]:
!wget https://github.com/PicPay/case-machine-learning-engineer-pleno/raw/main/notebook/airports-database.zip
!unzip airports-database.zip -d airports-database

--2024-08-25 23:09:40--  https://github.com/PicPay/case-machine-learning-engineer-pleno/raw/main/notebook/airports-database.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PicPay/case-machine-learning-engineer-pleno/main/notebook/airports-database.zip [following]
--2024-08-25 23:09:40--  https://raw.githubusercontent.com/PicPay/case-machine-learning-engineer-pleno/main/notebook/airports-database.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10463368 (10.0M) [application/zip]
Saving to: ‘airports-database.zip’

airports-database.z 100%[===================>]   9.98M  --.-KB/s    in 0.1s 

# Sessão

In [7]:
# iniciar uma sessão local
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, count, date_format, stddev, expr, first, to_date

#sc = SparkSession.builder.master('local[*]').getOrCreate()
spark = SparkSession.builder.appName("Introducao").getOrCreate()
spark

In [8]:
df = spark.read.csv("/content/airports-database/airports-database.csv", header=True)
df = df.withColumn("date", to_date(col("time_hour"), "yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("day_of_week_name", date_format(col("date"), "EEEE"))

## Perguntas

### 1. Qual é o número total de voos no conjunto de dados?

In [9]:
total_voos = df.count()
print(f"Número de voos total é {total_voos}")

Número de voos total é 336776


### 2. Quantos voos foram cancelados? (Considerando que voos cancelados têm dep_time e arr_time nulos)

In [10]:
num_empty_rows = df.filter(col("dep_time").isNull() & col("arr_time").isNull()).count()
print(f"Número de voos cancelados {num_empty_rows}")

Número de voos cancelados 8255


### 3. Qual é o atraso médio na partida dos voos (dep_delay)?

In [11]:
average = df.agg(avg(col("dep_delay")).alias("average")).collect()[0]["average"]
print(f"Media atraso médio {average:.2f}")

Media atraso médio 12.64


### 4. Quais são os 5 aeroportos com maior número de pousos?

In [12]:
dest_count_df = df.groupBy("dest").agg(count("*").alias("count"))
result = dest_count_df.sort(col("count").desc()).limit(5)

print(f"5 Aeroportos com maior numero de pouso")
result.show()

5 Aeroportos com maior numero de pouso
+----+-----+
|dest|count|
+----+-----+
| ORD|17283|
| ATL|17215|
| LAX|16174|
| BOS|15508|
| MCO|14082|
+----+-----+



### 5. Qual é a rota mais frequente (par origin-dest)?

In [13]:
dest_count_df = df.groupBy("origin","dest").agg(count("*").alias("count"))
result = dest_count_df.sort(col("count").desc()).limit(1)

print(f"rota acom maior frequencia")
result.show()

rota acom maior frequencia
+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
+------+----+-----+



### 6. Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada?(Exiba também o tempo)

In [14]:
average = df.groupBy("name", "carrier").agg(avg(col("arr_delay")).alias("average"))
average.sort(col("average").desc()).limit(5).show()

+--------------------+-------+------------------+
|                name|carrier|           average|
+--------------------+-------+------------------+
|Frontier Airlines...|     F9|21.920704845814978|
|AirTran Airways C...|     FL|20.115905511811025|
|ExpressJet Airlin...|     EV| 15.79643108710965|
|  Mesa Airlines Inc.|     YV|15.556985294117647|
|SkyWest Airlines ...|     OO|11.931034482758621|
+--------------------+-------+------------------+



### 7. Qual é o dia da semana com maior número de voos?

In [15]:
result = df.groupBy("day_of_week_name").agg(count("*").alias("count"))
result = result.sort(col("count").desc()).limit(1)

In [16]:
print(f"Dia da semana com maior numero de voos")
result.show()

Dia da semana com maior numero de voos
+----------------+-----+
|day_of_week_name|count|
+----------------+-----+
|          Monday|50690|
+----------------+-----+



### 8. Qual o percentual mensal dos voos tiveram atraso na partida superior a 30 minutos?

In [17]:
voos_total = df.count()
voos_total_atrasados_30 = df.filter(df.dep_delay > 30).count()
porcentagem = voos_total_atrasados_30/voos_total*100

print(f"Porcentagem de voos atrasados mais que 30m na partida {porcentagem:.2f}\%")

Porcentagem de voos atrasados mais que 30m na partida 14.34\%


### 9. Qual a origem mais comum para voos que pousaram em Seattle (SEA)?

In [18]:
seattle_df = df.filter(df.dest == "SEA")
result = seattle_df.groupBy("origin", "dest").agg(count("*").alias("count"))

result = result.sort(col("count").desc()).limit(1)
result_value = result.select("origin").rdd.flatMap(list).collect()[0]

In [19]:
print(f"Origem que mais pousa em seattle é {result_value}")

Origem que mais pousa em seattle é JFK


###  10. Qual é a média de atraso na partida dos voos (dep_delay) para cada dia da semana?

In [20]:
result = df.groupBy("day_of_week_name").agg(avg(col("dep_delay")).alias("average"))
result.show()

+----------------+------------------+
|day_of_week_name|           average|
+----------------+------------------+
|       Wednesday|11.803512219083876|
|         Tuesday|10.631682565455652|
|          Friday| 14.69605749486653|
|        Thursday|16.148919990957108|
|        Saturday| 7.650502333676133|
|          Monday|14.778936729330908|
|          Sunday|11.589531801152422|
+----------------+------------------+



###  11. Qual é a rota que teve o maior tempo de voo médio (air_time)?

In [21]:
result = df.sort(col("air_time").desc()).limit(1)
result.select("origin", "dest", "air_time").show()

+------+----+--------+
|origin|dest|air_time|
+------+----+--------+
|   LGA| CLT|    99.0|
+------+----+--------+



### 12. Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?

In [22]:
group_origin_dest_df = df.groupBy("origin", "dest").agg(count("*").alias("total_count"))
df_max_count = group_origin_dest_df.groupBy("origin").agg(max("total_count").alias("max_count"))

In [23]:
df_result = group_origin_dest_df.join(
    df_max_count,
    (group_origin_dest_df.origin == df_max_count.origin) &
    (group_origin_dest_df.total_count == df_max_count.max_count)
)

In [24]:
df_result.show()

+------+----+-----------+------+---------+
|origin|dest|total_count|origin|max_count|
+------+----+-----------+------+---------+
|   EWR| ORD|       6100|   EWR|     6100|
|   JFK| LAX|      11262|   JFK|    11262|
|   LGA| ATL|      10263|   LGA|    10263|
+------+----+-----------+------+---------+



###  13. Quais são as 3 rotas que tiveram a maior variação no tempo médio de voo (air_time) ?

In [25]:
result = df.groupBy("origin", "dest").agg(stddev(col("air_time")).alias("std"))
result = result.sort(col("std").desc()).limit(3)
result.show()

+------+----+------------------+
|origin|dest|               std|
+------+----+------------------+
|   LGA| MYR| 25.32455988429677|
|   EWR| HNL| 21.26613546847427|
|   JFK| HNL|20.688824842787056|
+------+----+------------------+



### 14. Qual é a média de atraso na chegada para voos que tiveram atraso na partida superior a 1 hora?

In [26]:
voos_atrasados_df = df.filter(df.dep_delay > 60)
result = voos_atrasados_df.agg(avg(col("arr_delay")).alias("average"))
result.show()

+------------------+
|           average|
+------------------+
|119.04880549963919|
+------------------+



### 15. Qual é a média de voos diários para cada mês do ano?

In [27]:
number_of_day_df = df.withColumn("number_of_day", expr("day(last_day(date))"))


In [35]:
group_by_month_df = number_of_day_df \
                    .groupBy("month") \
                    .agg(
                        (count("*")/first("number_of_day")) \
                        .alias("average")
                    )

group_by_month_df.show()

+-----+-----------------+
|month|          average|
+-----+-----------------+
|    7|949.1935483870968|
|   11|908.9333333333333|
|    3|930.1290322580645|
|    8|946.0322580645161|
|    5|928.9032258064516|
|    6|941.4333333333333|
|    9|919.1333333333333|
|    1|871.0967741935484|
|   10|931.9032258064516|
|    4|944.3333333333334|
|   12|907.5806451612904|
|    2|891.1071428571429|
+-----+-----------------+



### 16. Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?

In [29]:
voos_atrasados_df = df.filter(df.arr_delay > 30)
result = voos_atrasados_df.groupBy("origin", "dest").agg(count("*").alias("count"))
result = result.sort(col("count").desc()).limit(3)
result.show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   LGA| ATL| 1563|
|   JFK| LAX| 1286|
|   LGA| ORD| 1188|
+------+----+-----+



### 17. Para cada origem, qual o principal destino?

In [30]:
group_origin_dest_df = df.groupBy("origin", "dest").agg(count("*").alias("total_count"))
df_max_count = group_origin_dest_df.groupBy("origin").agg(max("total_count").alias("max_count"))

In [31]:
df_result = group_origin_dest_df.join(
    df_max_count,
    (group_origin_dest_df.origin == df_max_count.origin) &
    (group_origin_dest_df.total_count == df_max_count.max_count)
)

In [32]:
df_result.show()

+------+----+-----------+------+---------+
|origin|dest|total_count|origin|max_count|
+------+----+-----------+------+---------+
|   EWR| ORD|       6100|   EWR|     6100|
|   JFK| LAX|      11262|   JFK|    11262|
|   LGA| ATL|      10263|   LGA|    10263|
+------+----+-----------+------+---------+

